In [1]:
import fiona

gpkg_path = "RastersVetorizados_v1.gpkg"

layers = fiona.listlayers(gpkg_path)

print("Camadas no GeoPackage:")
for layer in layers:
    print("-", layer)

Camadas no GeoPackage:
- 1985_vetorizado
- 1986_vetorizado
- 1987_vetorizado
- 1988_vetorizado
- 1989_vetorizado
- 1990_vetorizado
- 1991_vetorizado
- 1992_vetorizado
- 1993_vetorizado
- 1994_vetorizado
- 1995_vetorizado
- 1996_vetorizado
- 1997_vetorizado
- 1998_vetorizado
- 1999_vetorizado
- 2000_vetorizado
- 2001_vetorizado
- 2002_vetorizado
- 2003_vetorizado
- 2004_vetorizado
- 2005_vetorizado
- 2006_vetorizado
- 2007_vetorizado
- 2008_vetorizado
- 2009_vetorizado
- 2010_vetorizado
- 2011_vetorizado
- 2012_vetorizado
- 2013_vetorizado
- 2014_vetorizado
- 2015_vetorizado
- 2016_vetorizado
- 2017_vetorizado
- 2018_vetorizado
- 2019_vetorizado
- 2020_vetorizado
- 2021_vetorizado
- 2022_vetorizado
- 2023_vetorizado
- 2024_vetorizado


In [10]:
import geopandas as gpd

gdf = gpd.read_file(gpkg_path, layer="1985_vetorizado")
print(gdf.head())

   id             nome codigo_ibg      area_m2  area_km2  valor   ano  \
0  14  SACO DOS LIM�ES       None   7277371.34      7.28   21.0  1985   
1  22             SEDE       None   7882128.58      7.88   21.0  1985   
2  31         TRINDADE       None  26545689.47     26.55   21.0  1985   
3  14  SACO DOS LIM�ES       None   7277371.34      7.28   21.0  1985   
4  31         TRINDADE       None  26545689.47     26.55   21.0  1985   

                                            geometry  
0  MULTIPOLYGON (((7.43e+05 6.95e+06, 7.43e+05 6....  
1  MULTIPOLYGON (((7.44e+05 6.95e+06, 7.44e+05 6....  
2  MULTIPOLYGON (((7.43e+05 6.95e+06, 7.43e+05 6....  
3  MULTIPOLYGON (((7.43e+05 6.95e+06, 7.44e+05 6....  
4  MULTIPOLYGON (((7.44e+05 6.95e+06, 7.44e+05 6....  


In [11]:
gdf["area_feicao_m2"] = gdf.geometry.area

In [12]:
print(gdf.head())

   id             nome codigo_ibg      area_m2  area_km2  valor   ano  \
0  14  SACO DOS LIM�ES       None   7277371.34      7.28   21.0  1985   
1  22             SEDE       None   7882128.58      7.88   21.0  1985   
2  31         TRINDADE       None  26545689.47     26.55   21.0  1985   
3  14  SACO DOS LIM�ES       None   7277371.34      7.28   21.0  1985   
4  31         TRINDADE       None  26545689.47     26.55   21.0  1985   

                                            geometry  area_feicao_m2  
0  MULTIPOLYGON (((7.43e+05 6.95e+06, 7.43e+05 6....      397.338735  
1  MULTIPOLYGON (((7.44e+05 6.95e+06, 7.44e+05 6....   428448.437558  
2  MULTIPOLYGON (((7.43e+05 6.95e+06, 7.43e+05 6....     3154.223706  
3  MULTIPOLYGON (((7.43e+05 6.95e+06, 7.44e+05 6....      396.012602  
4  MULTIPOLYGON (((7.44e+05 6.95e+06, 7.44e+05 6....     3203.987398  


In [14]:
import pandas as pd

df_soma = (
    gdf
    .groupby("nome", as_index=False)
    .agg(
        area_calculada_m2=("area_feicao_m2", "sum"),
        area_referencia_m2=("area_m2", "first")
    )
)

df_soma["dif_m2"] = (
    df_soma["area_calculada_m2"] - df_soma["area_referencia_m2"]
)

df_soma["dif_percentual"] = (
    df_soma["dif_m2"] / df_soma["area_referencia_m2"] * 100
)

In [16]:
display(df_soma.sort_values("dif_percentual", ascending=False))

,nome,area_calculada_m2,area_referencia_m2,dif_m2,dif_percentual
6,INGLESES,1.945655e+07,19440261.54,16284.638222,0.083768
11,RIO VERMELHO,3.136070e+07,31335430.14,25266.722528,0.080633
1,CACHOEIRA DO BOM JESUS,3.022703e+07,30203152.02,23879.285774,0.079062
0,BARRA DA LAGOA,8.095773e+06,8089401.33,6372.011729,0.078770
7,LAGOA DA CONCEI��O,5.118861e+07,51150091.75,38514.791415,0.075298
9,RATONES,3.238608e+07,32362172.34,23911.728093,0.073888
3,CANASVIEIRAS,2.912585e+07,29104773.26,21081.457793,0.072433
13,SACO GRANDE,1.887726e+07,18863890.04,13370.639852,0.070880
2,CAMPECHE,3.456266e+07,34538330.49,24329.225736,0.070441
17,TRINDADE,2.656421e+07,26545689.47,18520.048310,0.069767


In [19]:
df_soma.to_csv('Diferenças nas áreas.csv')

In [17]:
import geopandas as gpd

gdf1995 = gpd.read_file(gpkg_path, layer="1995_vetorizado")
gdf1995["area_feicao_m2"] = gdf1995.geometry.area
df_1995 = (
    gdf1995
    .groupby("nome", as_index=False)
    .agg(
        area_calculada_m2=("area_feicao_m2", "sum"),
        area_referencia_m2=("area_m2", "first")
    )
)

df_1995["dif_m2"] = (
    df_1995["area_calculada_m2"] - df_1995["area_referencia_m2"]
)

df_1995["dif_percentual"] = (
    df_1995["dif_m2"] / df_1995["area_referencia_m2"] * 100
)

In [18]:
display(df_1995.sort_values("dif_percentual", ascending=False))

,nome,area_calculada_m2,area_referencia_m2,dif_m2,dif_percentual
6,INGLESES,1.945655e+07,19440261.54,16284.638222,0.083768
11,RIO VERMELHO,3.136070e+07,31335430.14,25266.722528,0.080633
1,CACHOEIRA DO BOM JESUS,3.022703e+07,30203152.02,23879.285774,0.079062
0,BARRA DA LAGOA,8.095773e+06,8089401.33,6372.011728,0.078770
7,LAGOA DA CONCEI��O,5.118861e+07,51150091.75,38514.791414,0.075298
9,RATONES,3.238608e+07,32362172.34,23911.728093,0.073888
3,CANASVIEIRAS,2.912585e+07,29104773.26,21081.457793,0.072433
13,SACO GRANDE,1.887726e+07,18863890.04,13370.639852,0.070880
2,CAMPECHE,3.456266e+07,34538330.49,24329.225736,0.070441
17,TRINDADE,2.656421e+07,26545689.47,18520.048310,0.069767
